In [1]:
import os
import utilities
from maths_objects import (
    Formulas, SiteHierarchies, DataSource, FrontMatter, FormulaTable, PageTabs, 
    FormulaTableType, FormulaTableTypeSimple, FormulaTableTypeCalculus, FormulaFile)
import pandas as pd
from collections import namedtuple

data_source = DataSource()
site_hierarchy = SiteHierarchies(data_source.site_hierarchies)
formulas_by_year = Formulas(data_source.formulas_by_year)
formulas_cumulative = Formulas(data_source.formulas_by_year_cumulative)

In [15]:
index_formulas = formulas_by_year.filter_by_function(lambda x: x['Category'] == 'Indices')

index_table = FormulaTable(index_formulas)
index_table.set_type(FormulaTableTypeSimple)

index_file = FormulaFile()
index_file.is_cumulative_by_year = True
index_file.add_formula_table(index_table)

index_file.path_in_hierarchy()

'NSW/Year 9/Formulas/By year cumulative/Indices'

In [2]:
def create_formula_pages(table_type: FormulaTableType , 
                         group_by: list,
                         formulas: Formulas, 
                         base_dir: str,
                         hierarchies: SiteHierarchies,
                         is_cumulative_by_year:bool=False):
    """Creates formula pages"""

    for formula_group in formulas.group_by_columns(group_by):
        formula_table = FormulaTable(formula_group)
        formula_table.set_type(table_type)
        if formula_table.contains_content():
            formula_file = FormulaFile()
            formula_file.is_cumulative_by_year = is_cumulative_by_year
            formula_file.add_formula_table(formula_table)
            formula_file.set_weight_based_on_hierarchies(hierarchies)
            display(formula_file.path_in_hierarchy())
            formula_file.save(base_dir)



create_formula_pages(table_type=FormulaTableTypeSimple, 
                     group_by=['State', 'Subject', 'Category'],
                     formulas=formulas_by_year, 
                     base_dir=data_source.docs_directory,
                     hierarchies=site_hierarchy,
                     is_cumulative_by_year=False)



'NSW/Year 11 Adv/Formulas/By year/Differentiation'

'NSW/Year 11 Adv/Formulas/By year/Logarithms'

'NSW/Year 12 Adv/Formulas/By year/Differentiation'

'NSW/Year 12 Adv/Formulas/By year/Financial mathematics'

'NSW/Year 12 Adv/Formulas/By year/Integration'

'NSW/Year 12 Adv/Formulas/By year/Statistics'

'NSW/Year 12 Ext 1/Formulas/By year/Differentiation'

'NSW/Year 12 Ext 1/Formulas/By year/Integration'

'NSW/Year 9/Formulas/By year/Indices'

'NSW/Year 9/Formulas/By year/Surds'